In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Ratings and Mapping
raw_ratings = pd.read_excel('S&P Ratings - Quarterly.xlsx')
#IMF - ISO
imf_iso = pd.read_excel('Mappings.xlsx', sheet_name = 'IMF - ISO')
imf_iso_mapping = dict(zip(imf_iso.iloc[:, 0], imf_iso.iloc[:, 1]))
#WB - ISO
sp_iso = pd.read_excel('Mappings.xlsx', sheet_name = 'S&P - ISO')
sp_iso_mapping = dict(zip(sp_iso.iloc[:, 0], sp_iso.iloc[:, 1]))
#S&P WDI
sp_wdi = pd.read_excel('Mappings.xlsx', sheet_name = 'SP - WDI')
#Ratings
ratings_int = pd.read_excel('Mappings.xlsx', sheet_name = 'Ratings')
ratings_mapping = dict(zip(ratings_int.iloc[:, 0], ratings_int.iloc[:, 1]))
ratings_reverse = {b:a for a, b in ratings_mapping.items()}
#Ratings No modifiers
ratings_int_NM = pd.read_excel('Mappings.xlsx', sheet_name = 'Ratings_NoMod')
ratings_mapping_NM = dict(zip(ratings_int_NM.iloc[:, 0], ratings_int_NM.iloc[:, 1]))
ratings_reverse_NM = {b:a for a, b in ratings_mapping_NM.items() if a[-1] not in ['-','+']}
#Exclude
exclusions = ['Advanced Economies', 'Emerging and Developing Asia','Emerging and Developing Countries', 'Sub-Saharan Africa',
       'Western Hemisphere', 'World']
next_mapping = {"Q1":"Q2", "Q2":"Q3","Q3":"Q4","Q4":"Q1"}
prev_mapping = {"Q1":"Q4", "Q2":"Q1","Q3":"Q2","Q4":"Q3"}
dec_mapping = {25: 'Q1', 5: 'Q2', 75: 'Q3', 0: 'Q4'}
quarter_mapping = {'Q1': '0.25', 'Q2': '0.50', 'Q3': '0.75', 'Q4': '1.0'}

In [3]:
def Filter_DF(df = None, col = None, val = None, neg = False):
    data = df.copy()
    #data.replace(np.nan, "", inplace = True)
    if neg == False:
        if type(col) == str:
            if type(val) == str:
                return data[data[col] == val]
            elif type(val) == list:
                return data[data[col].isin(val)]
        elif type(col) == list:
            assert type(val) == list, "Multiple columns selected. Values should be a list"
            for i in range(len(col)):
                if type(val[i]) == str:
                    data = data[data[col[i]] == val[i]]
                elif type(val[i]) == list:
                    data = data[data[col[i]].isin(val[i])]
            return data
        
    if neg == True:
        if type(col) == str:
            if type(val) == str:
                return data[data[col] != val]
            elif type(val) == list:
                return data[~data[col].isin(val)]
        elif type(col) == list:
            assert type(val) == list, "Multiple columns selected. Values should be a list"
            for i in range(len(col)):
                if type(val[i]) == str:
                    data = data[data[col[i]] != val[i]]
                elif type(val[i]) == list:
                    data = data[~data[col[i]].isin(val[i])]
            return data

def WB_data(path):
    data = pd.read_excel(path)
    df = data[data['Country Code'].notnull()]
    del df['Country Name']
    del df['Series Code']
    df = df.replace({"..": np.nan})
    df.iloc[:, 2:] = df.iloc[:,2:].astype('float64')
    return df

def WB_interpolate(path, method = 'cubicspline'):
    mapping = {"03": "Q1", "06":"Q2", "09":"Q3", "12":"Q4"}
    WDI_GDP_Raw = pd.read_excel(path).replace({'..':np.nan})
    WDI_GDP_1 = WDI_GDP_Raw.transpose()
    WDI_GDP_val = WDI_GDP_1.iloc[4:, :]
    WDI_GDP_val.index = pd.to_datetime(WDI_GDP_val.index, format = '%Y')
    dt_ind = []
    for i in WDI_GDP_val.index:
        dt_ind.append(i.replace(month = 12, day = 31))
    WDI_GDP_val.index = dt_ind
    WDI_GDP_val2 = WDI_GDP_val.resample('Q').asfreq()
    WDI_GDP_val2 = WDI_GDP_val2.astype('float64')
    WDI_GDP_val3 = WDI_GDP_val2.copy()
    n = WDI_GDP_val3.shape[1]
    for i in range(n):
        if WDI_GDP_val3.iloc[:, i].notnull().sum() in [0, 1]:
            pass
        else:
            WDI_GDP_val3.iloc[:, i] = WDI_GDP_val3.iloc[:, i].interpolate(method)
    #WDI_GDP_val3 = WDI_GDP_val3.interpolate(method)
    dt_ind2 = []
    for i in WDI_GDP_val3.index:
        year = str(i).split('-')[0]
        month = str(i).split('-')[1]
        quarter = mapping[month]
        temp_ind = year + ' ' + quarter
        dt_ind2.append(temp_ind)
    WDI_GDP_val3.index = dt_ind2
    WDI_GDP_2 = WDI_GDP_1.copy()
    WDI_GDP_3 = pd.DataFrame()
    WDI_GDP_3 = pd.concat([WDI_GDP_3, WDI_GDP_2.iloc[:4, :], WDI_GDP_val3])
    WDI_GDP_4 = WDI_GDP_3.transpose()
    del WDI_GDP_4['Country Name']
    del WDI_GDP_4['Series Code']
    return WDI_GDP_4

def IMF_data(path, series = None, mapping = None, drop_cols = None, convert_per_capita = False):
    data = pd.read_excel(path, skiprows = 6)
    df = data.copy()
    df2 = df.iloc[:, 1:]
    del df2['Scale']
    del df2['Base Year']
    if mapping != None:
        df2.replace(mapping, inplace = True)
    df2.insert(1, 'Series Name', series)
    if drop_cols != None:
        df2.drop(drop_cols, axis = 1, inplace = True)
    df3 = Filter_DF(df2, 'Country', exclusions, neg = True)
    df3 = df3.replace({"...": np.nan, '-': 0.0})
    df3.iloc[:, 2:] = df3.iloc[:, 2:].astype('float64')
    df_cols = df3.columns.tolist()
    df_cols_new1 = ['Country Code'] + [df_cols[1]]
    df_cols_new2 = [i[:4] + ' ' + i[4:] for i in df_cols[2:]]
    df_cols_new = df_cols_new1 + df_cols_new2
    df3.columns = df_cols_new
    return df3

def IMF_inf(CPI):
    df = CPI.copy()
    inflation = pd.DataFrame()
    cpi_current = df.iloc[:, 3:].values
    cpi_prev = df.iloc[:, 2:-1].values
    inf_temp = (cpi_current - cpi_prev)/cpi_prev
    df2 = df.copy()
    df2.iloc[:, 3:] = inf_temp
    df3 = df2.drop(df2.columns[2], axis=1)
    return df3

def prev_period(period):
    period_split = period.split()
    if period_split[1] == 'Q1':
        prev_period = str(int(period_split[0]) - 1) + ' ' + prev_mapping[period_split[1]]
    else:
        prev_period = period_split[0] + ' ' + prev_mapping[period_split[1]]
    return prev_period

In [4]:
#WB
WGI = WB_data('data\WGI.xlsx')
WDI = WB_data('data\WDI.xlsx')
WDI_Pop = WB_data('data\WDI - Population.xlsx')
WDI_GDP = WB_interpolate(r'data\WDI - GDP Capita (Orig).xlsx')
WDI_Multi = WB_interpolate(r'data\WDI - Multi.xlsx')
WDI_Multi_2 = WB_interpolate(r'data\WDI - Multi 2.xlsx')
WDI_Multi_3 = WB_interpolate(r'data\WDI - Multi 3.xlsx')

In [5]:
#IMF
Q12 = ['1996Q1', '1996Q2']
Q123 = ['1996Q1', '1996Q2', '1996Q3']

Unemployment_Rate = IMF_data(r'data\Unemployment Rate.xlsx',series='Unemployment Rate',mapping=imf_iso_mapping,drop_cols=Q123)
CPI = IMF_data(r'data\CPI.xlsx',series='Inflation CPI',mapping=imf_iso_mapping,drop_cols=Q12)
Inflation_CPI = IMF_inf(CPI)
PPI = IMF_data(r'data\PPI.xlsx',series='Inflation PPI',mapping=imf_iso_mapping,drop_cols=Q12)
Inflation_PPI = IMF_inf(PPI)
Current_Account = IMF_data(r'data\Current Account - Net.xlsx',series='Current Account',mapping=imf_iso_mapping,drop_cols=Q123)
Fin_Assets = IMF_data(r'data\Net Acquisition of Financial Assets.xlsx',series='Net Acquisition of Financial Assets',mapping=imf_iso_mapping,drop_cols=Q123)
Reserves = IMF_data(r'data\Total Reserves excluding Gold - USD.xlsx',series='Total Reserves excluding Gold - USD',mapping=imf_iso_mapping,drop_cols=Q123)
Yield = IMF_data(r'data\Treasury Bills - Percent per Annum.xlsx',series='TBills - Yield',mapping=imf_iso_mapping,drop_cols=Q123)

In [6]:
feature_list = [WGI, WDI_Multi_3, Yield, Inflation_CPI]

features = pd.DataFrame()
for f in feature_list:
    features = pd.concat([features, f])
features = features.sort_values(by=['Country Code', 'Series Name'])
features.reset_index(inplace = True, drop = True)
unique_features = features['Series Name'].unique()
unique_countries = features['Country Code'].unique()
p = len(unique_features)
c = len(unique_countries)
for country in unique_countries:
    temp_df = Filter_DF(features, 'Country Code', country)
    temp_ind = temp_df.index
    if len(temp_df) < p:
        features.drop(temp_ind, axis = 0, inplace = True)
features.reset_index(inplace = True, drop = True)
features

,Country Code,Series Name,1996 Q4,1997 Q1,1997 Q2,1997 Q3,1997 Q4,1998 Q1,1998 Q2,1998 Q3,...,2019 Q3,2019 Q4,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4
0,AGO,"Agriculture, forestry, and fishing, value adde...",7.026869,6.170805,6.421647,7.478814,9.002018,10.613408,12.034404,12.920511,...,7.812613,7.882625,8.189303,8.681183,9.293243,9.946544,10.554046,11.064891,11.402429,11.485905
1,AGO,"Central government debt, total (% of GDP)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGO,Control of Corruption: Estimate,-1.167702,-1.169295,-1.170889,-1.172483,-1.174076,-1.17567,-1.177263,-1.178857,...,-1.103711,-1.078114,-1.046928,-1.015742,-0.984557,-0.953371,-0.878865,-0.804358,-0.729852,-0.655346
3,AGO,Current account balance (% of GDP),43.395691,24.019641,8.749454,-2.943636,-11.551993,-17.644539,-22.12061,-25.596385,...,9.022828,7.412294,5.571246,3.797987,2.401919,1.735458,2.108362,3.835472,7.288111,12.787388
4,AGO,Expense (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.017531,16.684952,17.699586,19.121718,21.029592,23.466535,26.416807,30.023775,34.363795,39.457403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,ZMB,"Services, value added (% of GDP)",41.543524,40.924948,40.75313,40.961617,41.471849,42.183392,43.036698,43.954715,...,54.701244,54.603759,54.476751,54.297812,54.024022,53.616377,53.049063,52.266894,51.223009,49.884158
2444,ZMB,TBills - Yield,58.1,46.166667,30.466667,22.066667,19.233333,17.733333,20.055333,29.289,...,21.896667,22.533333,23.203333,22.323333,19.136292,18.5432,18.952,18.856458,NaN,NaN
2445,ZMB,Trade (% of GDP),63.821482,62.460834,61.015971,59.581819,58.272863,57.208434,56.444557,56.088804,...,69.559938,68.791205,69.79716,72.246237,75.620863,79.325488,82.715927,85.355083,86.650517,85.992177
2446,ZMB,Trade in services (% of GDP),NaN,NaN,NaN,NaN,9.14649,9.498343,9.896307,10.322368,...,10.322166,10.922205,12.020235,13.378233,14.676378,15.557197,15.685192,14.746374,12.367524,8.209007


In [7]:
countries = features['Country Code'].unique()
periods = features.columns[2:].tolist()
first_period = periods[0]
last_period = periods[-1]
last_split = periods[-1].split()
if last_split[1] == 'Q4':
    next_period = str(int(last_split[0]) + 1) + ' ' + next_mapping[last_split[1]]
else:
    next_period = str(last_split[0]) + ' ' + next_mapping[last_split[1]]
periods_extra = periods + [next_period]

In [8]:
def encode_data(df, first_period = first_period, last_period = last_period, periods = periods, countries = countries, 
                lag = 5, standardize = True):
    lag_period = periods[lag]
    test_data = df.copy()
    encoded_data = pd.DataFrame()
    if last_split[1] == 'Q4':
        next_period = str(int(last_split[0]) + 1) + ' ' + next_mapping[last_split[1]]
    else:
        next_period = str(last_split[0]) + ' ' + next_mapping[last_split[1]]
    periods_extra = periods + [next_period]
    
    
    for p in range(len(periods) - (lag - 1)):
        period_list = [periods[p+j] for j in range(lag)]
        to_concat = test_data[['Country Code', 'Series Name'] + period_list]
        to_concat.columns = ['Country Code', 'Series Name'] + ['T-' + str(j) for j in reversed(range(1, lag+1))]
        to_concat.insert(2, 'Period', periods_extra[p+lag])
        encoded_data = pd.concat([encoded_data, to_concat])
    encoded_data_clean = encoded_data.replace({'...':np.nan, '..':np.nan})
    encoded_data_clean = encoded_data_clean.set_index(['Country Code', 'Period'])
    encoded_data_test = encoded_data_clean.copy()
    encoded_data_test.sort_index(inplace = True)
    for country in countries:
        for p in range(lag, len(periods_extra)):
            if encoded_data_test.loc[(country, periods_extra[p])].isna().any().any() == True:
                encoded_data_clean = encoded_data_clean.drop((country, periods_extra[p]))
                
    encoded_data = encoded_data_clean.reset_index()
    if standardize == False:
        return encoded_data
    else:
        data_mean, data_std = get_center(encoded_data)
        standardized_data = standardize_data(encoded_data, data_mean, data_std, lag)
        standardized_data.sort_values(by = ["Country Code", "Period", "Series Name"], inplace = True)
        return standardized_data
    
def get_center(encoded_df):
    reset_data = encoded_df.copy()
    unique_df = pd.DataFrame()
    for i in reset_data.columns.tolist()[3:]:
        lag = i.split('-')[1]
        temp_decoded_data = reset_data[['Country Code', 'Series Name', i]]
        
        lag = int(lag)
        def apply_lag(date, lag = lag):
            y, q = date.split()
            y = int(y)
            q = float(quarter_mapping[q])
            period = y + q
            period_lagged = period - (lag * 0.25)
            if (period_lagged % 1) == 0:
                period_lagged -= 1
            period_lagged = str(period_lagged)
            y_new, q_new = period_lagged.split('.')
            q_new = dec_mapping[int(q_new)]
            period_lagged_str = y_new + ' ' + q_new
            return period_lagged_str
        
        temp_decoded_data['Period'] = reset_data.apply(lambda x: apply_lag(x.Period), axis=1)
        temp_decoded_data.columns = ['Country Code', 'Series Name', 'Value', 'Period']
        unique_df = pd.concat([unique_df, temp_decoded_data])
    unique_df = unique_df.drop_duplicates()
    data_mean = unique_df.groupby('Series Name').mean()
    data_mean.reset_index(inplace = True)
    data_mean.columns = ['Series Name', 'Mean']
    
    data_std = unique_df.groupby('Series Name').agg(np.std, ddof=1)
    data_std.reset_index(inplace = True)
    del data_std['Country Code']
    del data_std['Period']
    data_std.columns = ['Series Name', 'Std']
    
    return data_mean, data_std

def standardize_data(data, data_mean, data_std, lag = 5):
    df = data.copy()
    df = df.merge(data_mean)
    df = df.merge(data_std)
    for i in range(0,lag):
        df.iloc[:, 3+i] = (df.iloc[:, 3+i] - df.loc[:, 'Mean'])/df.loc[:, 'Std']
    del df['Mean']
    del df['Std']
    
    return df

In [9]:
def encode_ratings(df, mapping = None, iso_codes = None):

    df2 = df.copy()
    n = df2.shape[1]
    
    if iso_codes != None:
        df2 = df2.rename(columns = {'Country': 'Country Code'})
        df2 = df2.replace(iso_codes)
        countries = df2[['Country Code']]
    else:
        countries = df2[['Country Name']]
    
    encoded_ratings = pd.DataFrame()
    cols = df2.columns.tolist()
    for i in range(2, n):
        temp_df = countries.copy()
        temp_df['Period'] = cols[i]
        temp_df['Ratings'] = df2.iloc[:, i]
        encoded_ratings = pd.concat([encoded_ratings, temp_df])
        
    if mapping is None:
        return encoded_ratings
    else:
        name_mapping = dict(zip(mapping.iloc[:, 0], mapping.iloc[:, 1]))
        encoded_ratings.replace(name_mapping, inplace = True)
        return encoded_ratings
    
def merge_rating(data, rating):
    merged_data = data.merge(rating, how = 'left', on = ['Country Code', 'Period'])
    merged_data['Ratings'] = merged_data['Ratings'].fillna('NR')
    merged_data = merged_data[merged_data['Ratings'] != "NR"]
    merged_data.sort_values(by=["Country Code", "Period", "Series Name"], inplace = True)
    merged_data.reset_index(drop = True, inplace = True)
    return merged_data

def to_pytorch(merged_data, mapping):
    x,y = merged_data.shape
    p = len(merged_data['Series Name'].unique())
    m = int(x/p)
    n = y - 4
    temp = pd.DataFrame()
    cols = ['Lag'] + list(merged_data.T.iloc[2, :p])
    labels = []
    for i in range(m):
        data_point =  merged_data.T.iloc[3:y-1, i*p:(i+1)*p]
        data_point.reset_index(inplace = True)
        data_point.columns = cols
        data_point.insert(0, 'Country Code', merged_data.T.iloc[0, i*p])
        data_point.insert(1, 'Period', merged_data.T.iloc[1, i*p])
        data_point.insert(2, 'Rating', merged_data.T.iloc[-1, i*p])
        labels.append(mapping[merged_data.T.iloc[-1, i*p]])
        temp = pd.concat([temp, data_point])

    data_final = temp.copy()
    data_final.iloc[:, 4:] = data_final.iloc[:, 4:].astype('float64')
    
    data_x = data_final.copy()
    data_y = pd.DataFrame(data = {'Labels': labels})
    
    return data_x, data_y

In [10]:
encoded_data = encode_data(features, lag = 5)
#encoded_data.to_excel('Encoded Data Q.xlsx', index = False)
#encoded_data = pd.read_excel('Encoded Data Q.xlsx')
encoded_ratings = encode_ratings(raw_ratings, iso_codes = sp_iso_mapping)

In [11]:
#Missing Ratings Check
merged_data = encoded_data.merge(encoded_ratings, how = 'left', on = ['Country Code', 'Period'])
missing_rating = merged_data[merged_data['Ratings'].isna()]
pd.unique(missing_rating['Country Code'])

array(['LSO', 'MDA', 'MDV', 'MUS', 'NAM', 'NPL', 'SLB'], dtype=object)

In [12]:
#Merge rating and data, extract dimensions
merged_data = merge_rating(encoded_data, encoded_ratings)

In [13]:
#Convert to data frame that can be converted to Pytorch tensors
data_x, data_y_NM = to_pytorch(merged_data, ratings_mapping_NM)
#data_x, data_y_M = to_pytorch(merged_data, ratings_mapping)

In [14]:
col_order = ['Country Code', 'Period', 'Rating',  'Lag', 
             'Agriculture, forestry, and fishing, value added (% of GDP)', 'Manufacturing, value added (% of GDP)', 
            'Services, value added (% of GDP)', 'Trade (% of GDP)', 'Trade in services (% of GDP)',
            'Central government debt, total (% of GDP)', 'Current account balance (% of GDP)', 
            'External balance on goods and services (% of GDP)', 'Foreign direct investment, net inflows (% of GDP)',
            'Foreign direct investment, net outflows (% of GDP)', 'Inflation CPI', 'TBills - Yield', 
            'GDP per capita (constant 2015 US$)', 'Expense (% of GDP)', 'Gross savings (% of GDP)', 
            'Exports of goods and services (% of GDP)', 'Imports of goods and services (% of GDP)', 
            'General government final consumption expenditure (% of GDP)', 'Control of Corruption: Estimate', 
            'Government Effectiveness: Estimate', 'Political Stability and Absence of Violence/Terrorism: Estimate', 
            'Regulatory Quality: Estimate', 'Rule of Law: Estimate', 'Voice and Accountability: Estimate']

data_x2 = data_x[col_order]

In [15]:
file_name = 'Data_XY_L5_v3.xlsx'
with pd.ExcelWriter(file_name) as writer:
    data_x2.to_excel(writer, sheet_name = "X", index = False)
    data_y_NM.to_excel(writer, sheet_name = "y_NM", index = False)
    #data_y_M.to_excel(writer, sheet_name = "y_M", index = False)

In [16]:
#v2 - Removed Short-Term Debt, Total Debt-Service, Total Reserves
#v3 - Fixed get_center function to correctly label period of data